In [2]:
### minimize training data ###
import pandas as pd
import random as rndn

filename = r"../input/train.csv"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 100000 #desired sample size
skip = sorted(rndn.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pd.read_csv(filename, skiprows=skip)
df.to_csv("../input/train_min.csv")


In [1]:
### https://towardsdatascience.com/deep-learning-using-pytorch-for-tabular-data-c68017d8b480 ###

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0,2010-03-27 02:26:39.0000001,14.5,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2
1,1,2015-02-15 22:41:32.0000003,9.0,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1
2,2,2011-01-14 18:10:00.000000129,5.7,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2
3,3,2011-06-20 14:52:22.0000004,18.5,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1
4,4,2011-10-14 12:20:00.00000051,4.9,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5


In [3]:
import pandas as pd

df_train = pd.read_csv("../input/train_min.csv")
df_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [4]:
df_test = pd.read_csv("../input/test.csv")
df_test.head()

0       2015-01-27 13:08:24.0000002
1       2015-01-27 13:08:24.0000003
2       2011-10-08 11:53:44.0000002
3       2012-12-01 21:12:12.0000002
4       2012-12-01 21:12:12.0000003
                   ...             
9909    2015-05-10 12:37:51.0000002
9910    2015-01-12 17:05:51.0000001
9911    2015-04-19 20:44:15.0000001
9912    2015-01-31 01:05:19.0000005
9913    2015-01-18 14:06:23.0000006
Name: key, Length: 9914, dtype: object

In [5]:
### data preprocessing ###

### copy for submission ###
df_test_for_sub = df_test["key"]
df_test_for_sub

1.5

In [6]:
# delete the fares that are less than 0
df_train = df_train.loc[df_train.fare_amount > 0]
df_train.fare_amount.min()

99990

In [7]:
len(df_train)

In [8]:
train_X = df_train.drop(columns=['fare_amount'])

Y = df_train['fare_amount']
#Y = np.log(df_train.fare_amount)

test_X = df_test

,Unnamed: 0,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,0.0,2010-03-27 02:26:39.0000001,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2
1,1.0,2015-02-15 22:41:32.0000003,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1
2,2.0,2011-01-14 18:10:00.000000129,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2
3,3.0,2011-06-20 14:52:22.0000004,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1
4,4.0,2011-10-14 12:20:00.00000051,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5


In [9]:
df = train_X.append(test_X, sort=False)
df.head()

In [10]:
### calculate the distance traveled ###

import numpy as np

def haversine_distance(df, lat1, long1, lat2, long2):
    """
    Calculates the haversine distance between 2 sets of GPS coordinates in df
    """
    r = 6371  # average radius of Earth in kilometers
       
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2]-df[lat1])
    delta_lambda = np.radians(df[long2]-df[long1])
     
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c) # in kilometers

    return d

,Unnamed: 0,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_km
0,0.0,2010-03-27 02:26:39.0000001,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2,4.618332
1,1.0,2015-02-15 22:41:32.0000003,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1,2.903933
2,2.0,2011-01-14 18:10:00.000000129,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2,0.962144
3,3.0,2011-06-20 14:52:22.0000004,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1,7.172769
4,4.0,2011-10-14 12:20:00.00000051,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5,0.912682


In [11]:
df['dist_km'] = haversine_distance(
    df,
    'pickup_latitude', 
    'pickup_longitude', 
    'dropoff_latitude', 
    'dropoff_longitude'
)

df.head()

,Unnamed: 0,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_km,EDTdate,Hour,AMorPM,Weekday
0,0.0,2010-03-27 02:26:39.0000001,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2,4.618332,2010-03-26 22:26:39,22,pm,Fri
1,1.0,2015-02-15 22:41:32.0000003,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1,2.903933,2015-02-15 18:41:32,18,pm,Sun
2,2.0,2011-01-14 18:10:00.000000129,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2,0.962144,2011-01-14 14:10:00,14,pm,Fri
3,3.0,2011-06-20 14:52:22.0000004,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1,7.172769,2011-06-20 10:52:22,10,am,Mon
4,4.0,2011-10-14 12:20:00.00000051,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5,0.912682,2011-10-14 08:20:00,8,am,Fri


In [12]:
### add a datetime column and derive useful statistics ###

df['EDTdate'] = pd.to_datetime(df['pickup_datetime'].str[:19]) - pd.Timedelta(hours=4)
df['Hour'] = df['EDTdate'].dt.hour
df['AMorPM'] = np.where(df['Hour']<12,'am','pm')
df['Weekday'] = df['EDTdate'].dt.strftime("%a")

df.head()

,Unnamed: 0,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_km,EDTdate,Hour,AMorPM,Weekday
0,0.0,2010-03-27 02:26:39.0000001,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2,4.618332,2010-03-26 22:26:39,22,pm,Fri
1,1.0,2015-02-15 22:41:32.0000003,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1,2.903933,2015-02-15 18:41:32,18,pm,Sun
2,2.0,2011-01-14 18:10:00.000000129,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2,0.962144,2011-01-14 14:10:00,14,pm,Fri
3,3.0,2011-06-20 14:52:22.0000004,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1,7.172769,2011-06-20 10:52:22,10,am,Mon
4,4.0,2011-10-14 12:20:00.00000051,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5,0.912682,2011-10-14 08:20:00,8,am,Fri


In [13]:
### prepare the model ###

### define categorical and continuous columns ###
cat_cols = [
    'Hour', 
    'AMorPM', 
    'Weekday'
]

cont_cols = [
    'pickup_latitude', 
    'pickup_longitude', 
    'dropoff_latitude', 
    'dropoff_longitude', 
    'passenger_count', 
    'dist_km'
]

df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109904 entries, 0 to 9913
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Unnamed: 0         99990 non-null   float64       
 1   key                109904 non-null  object        
 2   pickup_datetime    109904 non-null  object        
 3   pickup_longitude   109904 non-null  float64       
 4   pickup_latitude    109904 non-null  float64       
 5   dropoff_longitude  109903 non-null  float64       
 6   dropoff_latitude   109903 non-null  float64       
 7   passenger_count    109904 non-null  int64         
 8   dist_km            109903 non-null  float64       
 9   EDTdate            109904 non-null  datetime64[ns]
 10  Hour               109904 non-null  int64         
 11  AMorPM             109904 non-null  object        
 12  Weekday            109904 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(2), ob

In [14]:
df.info()

In [15]:
#make the cat categories as "category" and label encoder it
from sklearn import preprocessing

for col in df.columns:
  if col in cat_cols:
    df[col] = preprocessing.LabelEncoder().fit_transform(df[col])
    df[col]= df[col].astype('category')

,Unnamed: 0,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist_km,EDTdate,Hour,AMorPM,Weekday
0,0.0,2010-03-27 02:26:39.0000001,2010-03-27 02:26:39 UTC,-73.961598,40.716487,-73.960417,40.674963,2,4.618332,2010-03-26 22:26:39,22,1,0
1,1.0,2015-02-15 22:41:32.0000003,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1,2.903933,2015-02-15 18:41:32,18,1,3
2,2.0,2011-01-14 18:10:00.000000129,2011-01-14 18:10:00 UTC,-73.986370,40.747380,-73.975567,40.744572,2,0.962144,2011-01-14 14:10:00,14,1,0
3,3.0,2011-06-20 14:52:22.0000004,2011-06-20 14:52:22 UTC,-73.997892,40.724297,-73.977578,40.786941,1,7.172769,2011-06-20 10:52:22,10,0,1
4,4.0,2011-10-14 12:20:00.00000051,2011-10-14 12:20:00 UTC,-73.952797,40.780497,-73.955042,40.788527,5,0.912682,2011-10-14 08:20:00,8,0,0


In [16]:
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109904 entries, 0 to 9913
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Unnamed: 0         99990 non-null   float64       
 1   key                109904 non-null  object        
 2   pickup_datetime    109904 non-null  object        
 3   pickup_longitude   109904 non-null  float64       
 4   pickup_latitude    109904 non-null  float64       
 5   dropoff_longitude  109903 non-null  float64       
 6   dropoff_latitude   109903 non-null  float64       
 7   passenger_count    109904 non-null  int64         
 8   dist_km            109903 non-null  float64       
 9   EDTdate            109904 non-null  datetime64[ns]
 10  Hour               109904 non-null  category      
 11  AMorPM             109904 non-null  category      
 12  Weekday            109904 non-null  category      
dtypes: category(3), datetime64[ns](1), float64(6),

In [17]:
df.info()

Index(['Unnamed: 0', 'key', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'dist_km', 'EDTdate', 'Hour', 'AMorPM', 'Weekday'],
      dtype='object')

In [18]:
### take the relevent columns for the model and split back to train and test ###
df.columns

In [19]:
# keep only the cols for the model
df = df[
    [
        'Hour', 
        'AMorPM', 
        'Weekday',
        'pickup_latitude', 
        'pickup_longitude', 
        'dropoff_latitude', 
        'dropoff_longitude', 
        'passenger_count', 
        'dist_km'
    ]
]

df_train = df[:99994]
df_test = df[99994:]

<ipython-input-19-adfa01358160>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n] = (df[n]-means[n]) / (1e-7 + stds[n])


In [20]:
### continuous values ###

from pandas.api.types import is_numeric_dtype

# Compute the means and stds of `self.cont_names` columns to normalize them
def Normalize(df):
  means, stds = {}, {}
  cont_names = [
      'pickup_latitude', 
      'pickup_longitude', 
      'dropoff_latitude', 
      'dropoff_longitude', 
      'passenger_count', 
      'dist_km'
  ]
  
  for n in cont_names:
    assert is_numeric_dtype(df[n]), (f"""Cannot normalize '{n}' column as it isn't numerical. Are you sure it doesn't belong in the categorical set of columns?""")
    means[n],stds[n] = df[n].mean(),df[n].std()
    df[n] = (df[n]-means[n]) / (1e-7 + stds[n])


Normalize(df_train)
Normalize(df_test)
X = df_train

,Hour,AMorPM,Weekday,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,dist_km
0,22,1,0,0.126016,-0.133438,0.125627,-0.141685,0.246944,-0.040350
1,18,1,3,0.128596,-0.137225,0.130781,-0.146169,-0.522203,-0.045066
2,14,1,0,0.130813,-0.135680,0.136751,-0.143113,0.246944,-0.050408
3,10,0,1,0.127229,-0.136722,0.143521,-0.143302,-0.522203,-0.033323
4,8,0,0,0.135955,-0.132642,0.143775,-0.141179,2.554387,-0.050544


In [26]:
df_train.head()

(99990, 9)

In [27]:
X.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99990 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Hour               99990 non-null  category
 1   AMorPM             99990 non-null  category
 2   Weekday            99990 non-null  category
 3   pickup_latitude    99990 non-null  float64 
 4   pickup_longitude   99990 non-null  float64 
 5   dropoff_latitude   99989 non-null  float64 
 6   dropoff_longitude  99989 non-null  float64 
 7   passenger_count    99990 non-null  float64 
 8   dist_km            99989 non-null  float64 
dtypes: category(3), float64(6)
memory usage: 5.6 MB


In [28]:
X.info()

,Hour,AMorPM,Weekday,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,passenger_count,dist_km
58526,16,1,2,0.135145,-0.132729,0.135762,-0.143564,1.016092,-0.040195
38241,23,1,2,0.129786,-0.137378,0.132194,-0.134693,-0.522203,-0.024467
3806,23,1,4,0.129665,-0.137380,0.131407,-0.140626,-0.522203,-0.037359
27927,17,1,3,0.135288,-0.132924,0.143298,-0.141282,-0.522203,-0.050197
6006,19,1,2,0.133024,-0.135374,0.153770,-0.139646,-0.522203,-0.023840


In [29]:
### train-valid split ###
from sklearn import model_selection

X_train, X_val, y_train, y_val = model_selection.train_test_split(
    X, Y, test_size=0.20, random_state=42, shuffle=True
) 

X_train.head()

[(24, 12), (2, 1), (7, 4)]

In [30]:
### embeddings ###
cat_szs = [len(df[col].cat.categories) for col in cat_cols]
emb_szs = [(size, min(50, (size+1)//2)) for size in cat_szs]
emb_szs

list

In [31]:
type(emb_szs)

(79992, 9)

In [32]:
X_train.shape

(19998, 9)

In [33]:
X_val.shape

(79992,)

In [34]:
y_train.shape

(19998,)

In [37]:
y_val.shape

In [38]:
### model ###
from torch.utils.data import Dataset, DataLoader

class RegressionColumnarDataset(Dataset):
    def __init__(self, df, cats, y):
        self.dfcats = df[cats]
        self.dfconts = df.drop(cats, axis=1)
        self.cats = np.stack([c.values for n, c in self.dfcats.items()], axis=1).astype(np.int64)
        self.conts = np.stack([c.values for n, c in self.dfconts.items()], axis=1).astype(np.float32)
        self.y = y.values.astype(np.float32)
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return [self.cats[idx], self.conts[idx], self.y[idx]]


In [39]:
trainds = RegressionColumnarDataset(X_train, cat_cols, y_train) 
valds = RegressionColumnarDataset(X_val, cat_cols, y_val) 


In [41]:
params = {
    'batch_size': 128,
    'shuffle': True
}

traindl = DataLoader(trainds, **params)
valdl = DataLoader(valds, **params)

In [42]:
### tabular model ###
import torch
import torch.nn as nn

torch.manual_seed(33)

from collections.abc import Iterable


def bn_drop_lin(n_in:int, n_out:int, bn:bool=True, p:float=0., actn=None):
    "Sequence of batchnorm (if `bn`), dropout (with `p`) and linear (`n_in`,`n_out`) layers followed by `actn`."
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

def ifnone(a,b):
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

def listify(p, q):
    "Make `p` listy and the same length as `q`."
    if p is None: p=[]
    elif isinstance(p, str):          p = [p]
    elif not isinstance(p, Iterable): p = [p]
    #Rank 0 tensors in PyTorch are Iterable but don't have a length.
    else:
        try: a = len(p)
        except: p = [p]
    n = q if type(q)==int else len(p) if q is None else len(q)
    if len(p)==1: p = p * n
    assert len(p)==n, f'List len mismatch ({len(p)} vs {n})'
    return list(p)

              

class TabularModel(nn.Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs, n_cont:int, out_sz:int, layers, ps=None,
                 emb_drop:float=0., y_range=None, use_bn:bool=True, bn_final:bool=False):
        super().__init__()
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs]) #type: torch.nn.modules.container.ModuleList
        self.emb_drop = nn.Dropout(emb_drop) #type: torch.nn.modules.dropout.Dropout
        self.bn_cont = nn.BatchNorm1d(n_cont) #type torch.nn.modules.batchnorm.BatchNorm1d
        n_emb = sum(e.embedding_dim for e in self.embeds) # n_emb = 17 , type: int
        self.n_emb,self.n_cont,self.y_range = n_emb,n_cont,y_range
        sizes = [n_emb + n_cont] + layers + [out_sz] #typeL list, len: 4
        actns = [nn.ReLU(inplace=True) for _ in range(len(sizes)-2)] + [None] #type: list, len: 3.  the last in None because we finish with linear
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        if bn_final: layers.append(nn.BatchNorm1d(sizes[-1]))
        self.layers = nn.Sequential(*layers) #type: torch.nn.modules.container.Sequential
        



    def forward(self, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)] #take the embedding list and grab an embedding and pass in our single row of data.        
            x = torch.cat(x, 1) # concatenate it on dim 1 ## remeber that the len is the batch size
            x = self.emb_drop(x) # pass it through a dropout layer
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont) # batchnorm1d
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont # combine the categircal and continous variables on dim 1
        x = self.layers(x)
        if self.y_range is not None:
            x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0] # deal with y_range
        return x.squeeze()
    

(0, 312.264)

In [44]:
y_range = (0, y_train.max()*1.2)
y_range

device(type='cpu')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = TabularModel(emb_szs = emb_szs,n_cont = len(cont_cols),out_sz = 1,layers = [1000,500,250],ps= [0.001,0.01,0.01],emb_drop=0.04, y_range=y_range).to(device)
model

In [ ]:
### optimizer ###

from collections import defaultdict

opt = torch.optim.Adam(model.parameters(), lr=1e-2) # can add: weight_decay=
#lr_cosine = lr_scheduler.CosineAnnealingLR(opt,T_max = 350) #1000 ## if you want to add lr_cosine

lr = defaultdict(list)
tloss = defaultdict(list)
vloss = defaultdict(list)

In [ ]:
### train and fit ###

from tqdm import tqdm, tqdm_notebook, tnrange
from sklearn import metrics

def inv_y(y):
    return np.exp(y)

def rmse(targ, y_pred):
    return np.sqrt(metrics.mean_squared_error(inv_y(y_pred), inv_y(targ))) #.detach().numpy()
    

def rmse(targ, y_pred):
   return np.sqrt(metrics.mean_squared_error(y_pred, targ)) #.detach().numpy()



#def fit(model, train_dl, val_dl, loss_fn, opt, scheduler, epochs=3): ## if you want to add schedular
def fit(model, train_dl, val_dl, loss_fn, opt, epochs=3):
    num_batch = len(train_dl)
    for epoch in tnrange(epochs):      
        y_true_train = list()
        y_pred_train = list()
        total_loss_train = 0          
        
        t = tqdm_notebook(iter(train_dl), leave=False, total=num_batch)
        for cat, cont, y in t:
            cat = cat.cpu()
            cont = cont.cpu()
            y = y.cpu()
            
            t.set_description(f'Epoch {epoch}')
            
            opt.zero_grad() #find where the grads are zero
            pred = model(cat, cont)
            loss = loss_fn(pred, y)
            
            loss.backward() # do backprop
            lr[epoch].append(opt.param_groups[0]['lr'])
            tloss[epoch].append(loss.item())
            opt.step()
            #scheduler.step()
            
            
            t.set_postfix(loss=loss.item())
            
            y_true_train += list(y.cpu().data.numpy())
            y_pred_train += list(pred.cpu().data.numpy())
            total_loss_train += loss.item()
            
        train_acc = rmse(y_true_train, y_pred_train)
        train_loss = total_loss_train/len(train_dl) # len train_dl = 704. the calc is number of train examples (89991) / batch size (128)
        
        if val_dl:
            y_true_val = list()
            y_pred_val = list()
            total_loss_val = 0
            for cat, cont, y in tqdm_notebook(val_dl, leave=False):
                cat = cat.cpu()
                cont = cont.cpu()
                y = y.cpu()
                pred = model(cat, cont)
                loss = loss_fn(pred, y)
                
                y_true_val += list(y.cpu().data.numpy())
                y_pred_val += list(pred.cpu().data.numpy())
                total_loss_val += loss.item()
                vloss[epoch].append(loss.item())
            valacc = rmse(y_true_val, y_pred_val)
            valloss = total_loss_val/len(valdl)
            print(f'Epoch {epoch}: train_loss: {train_loss:.4f} train_rmse: {train_acc:.4f} | val_loss: {valloss:.4f} val_rmse: {valacc:.4f}')
        else:
            print(f'Epoch {epoch}: train_loss: {train_loss:.4f} train_rmse: {train_acc:.4f}')
    
    return lr, tloss, vloss


In [45]:
### train the model ###

#lr, tloss, vloss = fit(model=model, train_dl=traindl, val_dl=valdl, loss_fn=nn.MSELoss(), opt=opt, scheduler=lr_cosine, epochs=10)


lr, tloss, vloss = fit(
    model=model, 
    train_dl=traindl, 
    val_dl=valdl, 
    loss_fn=nn.MSELoss(), 
    opt=opt,  
    epochs=10
)


TabularModel(
  (embeds): ModuleList(
    (0): Embedding(24, 12)
    (1): Embedding(2, 1)
    (2): Embedding(7, 4)
  )
  (emb_drop): Dropout(p=0.04, inplace=False)
  (bn_cont): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=23, out_features=1000, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.001, inplace=False)
    (4): Linear(in_features=1000, out_features=500, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.01, inplace=False)
    (8): Linear(in_features=500, out_features=250, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.01, inplace=False)
    (12): Linear(in_features=250, out_features=1, bias=

In [46]:
### plots ###
#Plot the Learning rate

import matplotlib.pyplot as plt

_ = plt.plot(lr[0])
_ = plt.title('LR over Epoch (Epoch 0)')

In [57]:
#inspired from: https://nbviewer.jupyter.org/github/FraPochetti/KagglePlaygrounds/blob/master/NYC%20Taxi%20Fares%20Prediction.ipynb 
# Train VS Validaton Loss

t = [np.mean(tloss[el]) for el in tloss]
v = [np.mean(vloss[el]) for el in vloss]

plt.plot(t, label='Training loss')
plt.plot(v, label='Validation loss')
plt.title("Train VS Validation Loss over Epochs")
plt.xlabel("Epochs")
plt.legend(frameon=False)

In [ ]:
### test ###
df_test.info()

In [ ]:
# divide to cats and conts columns + make them a tensor + to.cuda()

df_test_cats = df_test[['Hour', 'AMorPM', 'Weekday']]
test_cats = df_test_cats.astype(np.int64)
test_cats = torch.tensor(test_cats.values).cpu()

df_test_conts = df_test[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'passenger_count', 'dist_km']]
test_conts = df_test_conts.astype(np.float32)
test_conts = torch.tensor(test_conts.values).cpu()

In [ ]:
# make prediction

with torch.no_grad():
    model.eval()
    output = model.forward(test_cats,test_conts).cpu()

In [ ]:
output

In [ ]:
len(output)

In [ ]:
type(output)

In [ ]:
# convert to numpy array
np_output = output.cpu().numpy()

In [ ]:
type(np_output)

In [ ]:
np_output

In [ ]:
#  exp to the output data

#np_output_exp = np.exp(np_output)

In [ ]:
#np_output_exp

In [ ]:
# CONVERT to pandas data frame 

df_output =  pd.DataFrame(np_output)

#df_output =  pd.DataFrame(np_output_exp)

In [ ]:
df_output

In [ ]:
type(df_output)

In [ ]:
#output file

df_output.to_csv('../input/submission.csv', index=False)

In [ ]:
# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)

from IPython.display import HTML
import base64

def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
# create a link to download the dataframe
create_download_link(df_output)